## blood, spleen, muscle RNA editing specific, high-confidence genes as determined by Gabay et al. using RediToolKnown.py

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from scipy import stats
import os
import seaborn as sns
import math



### blood

In [2]:
# get files from folder
bloodFolder="/Users/karlfrontzek/Documents/UCL/rna_editing/prionREDI/blood"
os.chdir(bloodFolder)
onlyfiles = [f for f in os.listdir(bloodFolder) if os.path.isfile(os.path.join(bloodFolder, f))]

In [3]:
# read files in dataframe
df = pd.DataFrame()
for i in range(0,len(onlyfiles)):
    fname=onlyfiles[i].split(".")[0] # assign sample name
    df0=pd.read_csv(onlyfiles[i], sep="\t") # read file
    df0['sampleID']=fname # assign sample to table
    df=df.append(df0) # append table

/Users/karlfrontzek/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
# split base counts
dfb=df["BaseCount[A,C,G,T]"].str.split(",",expand=True) # expand column
dfb.columns=['A',"C","G","T"] # rename columns
dfb['A']=dfb['A'].str.replace('[',"") # replace brackets
dfb['T']=dfb['T'].str.replace(']',"")

In [5]:
df[['A',"C","G","T"]]=dfb # merge tables

In [6]:
df[['A',"C","G","T"]]= df[['A',"C","G","T"]].astype("int")

In [7]:
# 4 rows with T>=3, but A as ref allele
# keep only G>=3
df_raw=df
df=df.loc[df["G"]>=3]

In [8]:
# merge metadata
os.chdir("/Users/karlfrontzek/Documents/UCL/rna_editing/prion/")
# read meta-data and extract ID
peri_meta=pd.read_csv("meta_p3506_PeripheralSamples.csv",sep=";")
peri_meta.columns=['TubeID','SampleID','Treatment','wpi','Region']
peri_meta=peri_meta.loc[peri_meta['Region']=="blood"]
peri_meta['SampleID']=peri_meta['SampleID'].str.split("_",expand=True)[0]

In [9]:
# calculate editing frequency
df['editFreq']=df['G']/df['A']
# sort by editing frequency >= 1%
df.loc[df['editFreq']>=0.01] # all values are > 1 %, no addtional exclusion of values needed

/Users/karlfrontzek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,AllSubs,"BaseCount[A,C,G,T]",Coverage-q30,Frequency,MeanQ,Position,Reference,Region,Strand,Unnamed: 0,sampleID,A,C,G,T,editFreq
0,AG,"[264, 0, 650, 0]",914.0,0.71,40.59,74226862.0,A,chrX,0.0,NaN,PB-79,264,0,650,0,2.462121
2,AG,"[11, 0, 15, 0]",26.0,0.58,43.04,17804344.0,A,chr13,0.0,NaN,PB-79,11,0,15,0,1.363636
18,AG,"[19, 0, 3, 0]",22.0,0.14,40.09,75719719.0,A,chr14,0.0,NaN,PB-79,19,0,3,0,0.157895
30,-,"[38, 0, 3, 0]",41.0,0.00,41.71,93189584.0,A,chr5,0.0,NaN,PB-79,38,0,3,0,0.078947
0,AG,"[300, 0, 1319, 0]",1619.0,0.81,37.29,74226862.0,A,chrX,0.0,NaN,PB-122,300,0,1319,0,4.396667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,AG,"[2, 0, 9, 0]",11.0,0.82,69.55,17804344.0,A,chr13,0.0,NaN,PB-74,2,0,9,0,4.500000
3,AG,"[13, 0, 3, 0]",16.0,0.19,70.94,17804548.0,A,chr13,0.0,NaN,PB-74,13,0,3,0,0.230769
13,-,"[97, 0, 5, 0]",102.0,0.00,61.55,91656089.0,A,chr16,1.0,NaN,PB-74,97,0,5,0,0.051546
17,AG,"[20, 0, 3, 0]",23.0,0.13,46.13,75719719.0,A,chr14,0.0,NaN,PB-74,20,0,3,0,0.150000


In [11]:
df=df.merge(peri_meta,left_on="sampleID",right_on="SampleID") # merge tables

In [12]:
# keep df_raw for statistical comparisons
df_raw=df_raw.merge(peri_meta,left_on="sampleID",right_on="SampleID") # merge tables

In [20]:
# test if edited site occurs at least in 2/3 of replicates, easy: positions are all unique
occurence = []
wpii=[4,4,8,8,12,12,14,14,16,16,18,18,20,20,'term','term']
treat=['RML6','NBH']*8
loopi=pd.DataFrame({"treat":treat,"wpi":wpii})


In [28]:
df.columns=['AllSubs', 'BaseCount[A,C,G,T]', 'Coverage-q30', 'Frequency', 'MeanQ',
       'Position', 'Reference', 'Region_x', 'Strand', 'Unnamed: 0', 'sampleID',
       'A', 'C', 'G', 'T', 'editFreq', 'TubeID_x', 'SampleID_x', 'Treatment_x',
       'wpi_x', 'Region_y', 'TubeID_y', 'SampleID', 'Treatment', 'wpi',
       'Region'] # rename SampleID, Treatment and wpi columns

In [23]:
df['Position'].loc[(df['wpi']==str(loopi['wpi'].iloc[0])) & (df['Treatment']==loopi['treat'].iloc[0])].value_counts()

74226862.0     3
17804344.0     3
91655615.0     2
91656089.0     2
75719719.0     2
172092348.0    2
17804548.0     2
91656122.0     1
91656061.0     1
91656133.0     1
93189584.0     1
157558149.0    1
17804365.0     1
Name: Position, dtype: int64

In [24]:
tS=pd.DataFrame([])
tS[['Position','Counts']]=pd.DataFrame(df['Position'].loc[(df['wpi']==str(loopi['wpi'].iloc[0])) & (df['Treatment']==loopi['treat'].iloc[0])].value_counts()).reset_index()


In [25]:
# extract sites that are edited at least in 2/3 replicates

testSites=pd.DataFrame([])
for i in range(0,len(loopi)):
    tS=pd.DataFrame([])
    tS[['Position','Counts']]=pd.DataFrame(df['Position'].loc[(df['wpi']==str(loopi['wpi'].iloc[i])) & (df['Treatment']==loopi['treat'].iloc[i])].value_counts()).reset_index()
    tS['wpi']=loopi['wpi'].iloc[i]
    tS['Treatment']=loopi['treat'].iloc[i]
    testSites=testSites.append(tS)

In [27]:
df

,AllSubs,"BaseCount[A,C,G,T]",Coverage-q30,Frequency,MeanQ,Position,Reference,Region_x,Strand,Unnamed: 0,...,TubeID_x,SampleID_x,Treatment_x,wpi_x,Region_y,TubeID_y,SampleID_y,Treatment,wpi,Region
0,AG,"[264, 0, 650, 0]",914.0,0.71,40.59,74226862.0,A,chrX,0.0,NaN,...,p1608_4212/44,PB-79,NBH,16,blood,p1608_4212/44,PB-79,NBH,16,blood
1,AG,"[11, 0, 15, 0]",26.0,0.58,43.04,17804344.0,A,chr13,0.0,NaN,...,p1608_4212/44,PB-79,NBH,16,blood,p1608_4212/44,PB-79,NBH,16,blood
2,AG,"[19, 0, 3, 0]",22.0,0.14,40.09,75719719.0,A,chr14,0.0,NaN,...,p1608_4212/44,PB-79,NBH,16,blood,p1608_4212/44,PB-79,NBH,16,blood
3,-,"[38, 0, 3, 0]",41.0,0.00,41.71,93189584.0,A,chr5,0.0,NaN,...,p1608_4212/44,PB-79,NBH,16,blood,p1608_4212/44,PB-79,NBH,16,blood
4,AG,"[300, 0, 1319, 0]",1619.0,0.81,37.29,74226862.0,A,chrX,0.0,NaN,...,p1608_5588/78,PB-122,RML6,term,blood,p1608_5588/78,PB-122,RML6,term,blood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,AG,"[2, 0, 9, 0]",11.0,0.82,69.55,17804344.0,A,chr13,0.0,NaN,...,p1608_5588/74,PB-74,RML6,16,blood,p1608_5588/74,PB-74,RML6,16,blood
559,AG,"[13, 0, 3, 0]",16.0,0.19,70.94,17804548.0,A,chr13,0.0,NaN,...,p1608_5588/74,PB-74,RML6,16,blood,p1608_5588/74,PB-74,RML6,16,blood
560,-,"[97, 0, 5, 0]",102.0,0.00,61.55,91656089.0,A,chr16,1.0,NaN,...,p1608_5588/74,PB-74,RML6,16,blood,p1608_5588/74,PB-74,RML6,16,blood
561,AG,"[20, 0, 3, 0]",23.0,0.13,46.13,75719719.0,A,chr14,0.0,NaN,...,p1608_5588/74,PB-74,RML6,16,blood,p1608_5588/74,PB-74,RML6,16,blood


In [29]:
dfReplicates=df[['wpi','Treatment','SampleID']].drop_duplicates()
totalRep=pd.Series(list(zip(dfReplicates.wpi, dfReplicates.Treatment))).value_counts()
totalRep=totalRep.reset_index()
totalRep.columns=['Index','Rep']

In [30]:
testSites['wpi']=testSites['wpi'].astype('str')

In [31]:
testSites['wpiTreatment']=list(zip(testSites.wpi, testSites.Treatment)) # merge wpi+Treatment for threshold 
Thr=pd.DataFrame({'Index':[],'Thr':[]})                                                       # calculation of edited sites
for i in range(0,len(testSites)): # iterate over testsite counts, keep those occuring in floor(2/3 of samples)
    wpiTreat=testSites['wpiTreatment'].iloc[i]  # read current treatment+wpi combi
    Tr=math.floor(totalRep['Rep'].loc[totalRep['Index']==wpiTreat]*2/3) # calculate current threshold
    Tp=pd.DataFrame({'Index':str(i),'Thr':str(Tr)},index=[i]) # make list of thresholds per edited site
    Thr=Thr.append(Tp)
#testSitesThr=testSites['Position'].loc[testSites['Counts']!=1] # extract test sites 
#testSites['Counts'].loc[testSites['Position'].isin(testSites1)].head(50)

In [32]:
testSitesThr=testSites.reset_index().join(Thr)# join TestSites and Threshold

In [33]:
testSitesThr['Counts']=testSitesThr['Counts'].astype('int')
testSitesThr['Thr']=testSitesThr['Thr'].astype('int') # convert counts to int

In [34]:
testSitesThr=testSitesThr.loc[testSitesThr['Counts']>=testSitesThr['Thr']] # select for editing events which pass threshold

In [35]:
# make combination of position+wpi
uniqueEdSitesThr=pd.Series(list(zip(testSitesThr.wpi, testSitesThr.Position)))
uniqueEdSitesThr=uniqueEdSitesThr.astype('str') # convert tuple to str

In [36]:
df_raw['wpiPosition']=list(zip(df_raw.wpi, df_raw.Position)) # combine wpi+Position in df_raw
df_raw['wpiPosition']=df_raw['wpiPosition'].astype('str') # convert tuple to str

In [37]:
testSitesThr['wpiPosition']=testSitesThr['wpi']+'+'+testSitesThr['Position'].astype('str')
df_raw['wpiPosition']=df_raw['wpi']+'+'+df_raw['Position'].astype('str')

In [38]:
uniqueTestSitesThr=testSitesThr['wpiPosition'].unique() # retrieve unique editing sites

In [39]:
uniqueTestSitesThr=list(uniqueTestSitesThr) # convert unique editing sites to list

In [40]:
dfThr=df_raw.loc[df_raw['wpiPosition'].isin(uniqueTestSitesThr)] # keep editing sites that pass threshold

In [41]:
cog3=df_raw.loc[(df_raw['Position']==75719719) & (df_raw['wpi']=="term")]#write raw cog3 editing events
cog3.to_csv('cog3_editing.csv')

In [42]:
# clean up dfThr for export
dfThrClean=dfThr[['A','G','Treatment','wpi','wpiPosition']]


In [43]:
# test whether at least 2 replicates of non-edited site exist
uniqueSites=dfThrClean['wpiPosition'].unique()
excludeSites=[]
for i in range(0,len(uniqueSites)):
    uS=dfThrClean.loc[dfThrClean['wpiPosition']==uniqueSites[i]]
    if len(uS['Treatment'].loc[uS['Treatment']=="NBH"]) < 2:
        #print("In wpiPosition {} treated with NBH less than 2 replicates".format(uniqueSites[i]))
        excludeSites.append(uniqueSites[i])
    if len(uS['Treatment'].loc[uS['Treatment']=="RML6"]) < 2:
        #print("In wpiPosition {} treated with RML6 less than 2 replicates".format(uniqueSites[i]))
         excludeSites.append(uniqueSites[i])

In [44]:
dfThrClean=dfThrClean.loc[~dfThrClean['wpiPosition'].isin(excludeSites)]
dfThrClean.to_csv('bloodRediEli.csv') # export dfThrClean

#### analyse csv using R, see README.md
#### import csv from R below

In [45]:
# analyse statistical testing with Redit, see R
dfP=pd.read_csv('/Users/karlfrontzek/Documents/UCL/rna_editing/prionREDI/bloodEditingSitesRedit.csv')

In [46]:
dfP[['wpi','site1']]=dfP['site'].str.split("+",expand=True)

In [47]:
from statsmodels.stats.multitest import multipletests # for fdr

In [48]:
wpiUnique=dfP['wpi'].unique()

In [49]:
# adjust for fdr < 0.05 
# preassign dataframe
tT=pd.DataFrame({'wpi':[],'sig':[],'fdr':[],'pValue':[],'index':[]})
for i in range(0,len(wpiUnique)): # loop through all wpi
    tTtemp=pd.DataFrame({'wpi':[],'sig':[],'fdr':[],'pValue':[],'index':[]}) # preassign temp datafram
    # add sig and pVal to dataframe
    tTtemp['sig']=multipletests(dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]],alpha=0.05,method="fdr_bh")[0]
    tTtemp['fdr']=multipletests(dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]],alpha=0.05,method="fdr_bh")[1]
    tTtemp[['index','pValue']]=dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]].reset_index()
    tTtemp['wpi']=wpiUnique[i]
    tT=tT.append(tTtemp)

In [50]:
bloodRediAllEvents=tT.merge(dfP,left_on="pValue",right_on="pValue") # merge sites and fdr on "pValue"

In [51]:
bloodRediAllEvents.loc[bloodRediAllEvents['sig']==1] # show significant editing events

,wpi_x,sig,fdr,pValue,index,site,wpi_y,site1
12,term,1.0,0.002927,0.000418,12.0,term+75719719.0,term,75719719.0


# muscle

In [52]:
# get files from folder
muscleFolder="/Users/karlfrontzek/Documents/UCL/rna_editing/prionREDI/muscle"
os.chdir(muscleFolder)
onlyfiles = [f for f in os.listdir(muscleFolder) if os.path.isfile(os.path.join(muscleFolder, f))]

In [53]:
# read files in dataframe
df = pd.DataFrame()
df0 = pd.DataFrame()
for i in range(0,len(onlyfiles)):
    fname=onlyfiles[i].split(".")[0] # assign sample name
    df0=pd.read_csv(onlyfiles[i], sep="\t") # read file
    df0['sampleID']=fname # assign sample to table
    df=df.append(df0) # append table

In [54]:
# split base counts
dfb=df["BaseCount[A,C,G,T]"].str.split(",",expand=True) # expand column
dfb.columns=['A',"C","G","T"] # rename columns
dfb['A']=dfb['A'].str.replace('[',"") # replace brackets
dfb['T']=dfb['T'].str.replace(']',"")

In [55]:
df[['A',"C","G","T"]]=dfb # merge tables

In [56]:
df[['A',"C","G","T"]]= df[['A',"C","G","T"]].astype("int")

In [57]:
# 4 rows with T>=3, but A as ref allele
# keep only G>=3
df_raw=df
df=df.loc[df["G"]>=3]

In [58]:
# merge metadata
os.chdir("/Users/karlfrontzek/Documents/UCL/rna_editing/prion/")
# read meta-data and extract ID
peri_meta=pd.read_csv("meta_p3506_PeripheralSamples.csv",sep=";")
peri_meta.columns=['TubeID','SampleID','Treatment','wpi','Region']
peri_meta=peri_meta.loc[peri_meta['Region']=="muscle"]
peri_meta['SampleID']=peri_meta['SampleID'].str.split("_",expand=True)[0]

In [59]:
# calculate editing frequency
df['editFreq']=df['G']/df['A']
# sort by editing frequency >= 1%
#len(df['editFreq'])
len(df.loc[df['editFreq']>=0.01]) # all values are > 1 %

/Users/karlfrontzek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


307

In [60]:
df=df.merge(peri_meta,left_on="sampleID",right_on="SampleID") # merge tables

In [61]:
# keep df_raw for statistical comparisons
df_raw=df_raw.merge(peri_meta,left_on="sampleID",right_on="SampleID") # merge tables

In [62]:
# test if edited site occurs at least in 2/3 of replicates, easy: positions are all unique
occurence = []
wpii=[4,4,8,8,12,12,14,14,16,16,18,18,20,20,'term','term']
treat=['RML6','NBH']*8
loopi=pd.DataFrame({"treat":treat,"wpi":wpii})

In [63]:
# extract sites that are edited at least in 2/3 replicates

testSites=pd.DataFrame([])
for i in range(0,len(loopi)):
    tS=pd.DataFrame([])
    tS[['Position','Counts']]=pd.DataFrame(df['Position'].loc[(df['wpi']==str(loopi['wpi'].iloc[i])) & (df['Treatment']==loopi['treat'].iloc[i])].value_counts()).reset_index()
    tS['wpi']=loopi['wpi'].iloc[i]
    tS['Treatment']=loopi['treat'].iloc[i]
    testSites=testSites.append(tS)

In [64]:
dfReplicates=df[['wpi','Treatment','SampleID']].drop_duplicates()
totalRep=pd.Series(list(zip(dfReplicates.wpi, dfReplicates.Treatment))).value_counts()
totalRep=totalRep.reset_index()
totalRep.columns=['Index','Rep']

In [65]:
testSites['wpi']=testSites['wpi'].astype('str')

In [66]:
testSites['wpiTreatment']=list(zip(testSites.wpi, testSites.Treatment)) # merge wpi+Treatment for threshold 
Thr=pd.DataFrame({'Index':[],'Thr':[]})                                                       # calculation of edited sites
for i in range(0,len(testSites)): # iterate over testsite counts, keep those occuring in floor(2/3 of samples)
    wpiTreat=testSites['wpiTreatment'].iloc[i]  # read current treatment+wpi combi
    Tr=math.floor(totalRep['Rep'].loc[totalRep['Index']==wpiTreat]*2/3) # calculate current threshold
    Tp=pd.DataFrame({'Index':str(i),'Thr':str(Tr)},index=[i]) # make list of thresholds per edited site
    Thr=Thr.append(Tp)
#testSitesThr=testSites['Position'].loc[testSites['Counts']!=1] # extract test sites 
#testSites['Counts'].loc[testSites['Position'].isin(testSites1)].head(50)

In [67]:
testSitesThr=testSites.reset_index().join(Thr)# join TestSites and Threshold

In [68]:
testSitesThr['Counts']=testSitesThr['Counts'].astype('int')
testSitesThr['Thr']=testSitesThr['Thr'].astype('int') # convert counts to int

In [69]:
testSitesThr=testSitesThr.loc[testSitesThr['Counts']>=testSitesThr['Thr']] # select for editing events which pass threshold

In [70]:
# make combination of position+wpi
uniqueEdSitesThr=pd.Series(list(zip(testSitesThr.wpi, testSitesThr.Position)))
uniqueEdSitesThr=uniqueEdSitesThr.astype('str') # convert tuple to str

In [71]:
df_raw['wpiPosition']=list(zip(df_raw.wpi, df_raw.Position)) # combine wpi+Position in df_raw
df_raw['wpiPosition']=df_raw['wpiPosition'].astype('str') # convert tuple to str

In [72]:
testSitesThr['wpiPosition']=testSitesThr['wpi']+'+'+testSitesThr['Position'].astype('str')
df_raw['wpiPosition']=df_raw['wpi']+'+'+df_raw['Position'].astype('str')

In [73]:
uniqueTestSitesThr=testSitesThr['wpiPosition'].unique() # retrieve unique editing sites

In [74]:
uniqueTestSitesThr=list(uniqueTestSitesThr) # convert unique editing sites to list

In [75]:
df_raw.loc[df_raw['wpiPosition']=="4+77407731"]

,Region_x,Position,Reference,Strand,Coverage-q30,MeanQ,"BaseCount[A,C,G,T]",AllSubs,Frequency,sampleID,A,C,G,T,TubeID,SampleID,Treatment,wpi,Region_y,wpiPosition
1122,chr5,77407731,A,0,11,55.18,"[2, 0, 9, 0]",AG,0.82,PB-11,2,0,9,0,p1608_4211/55,PB-11,RML6,4,muscle,4+77407731


In [76]:
dfThr=df_raw.loc[df_raw['wpiPosition'].isin(uniqueTestSitesThr)] # keep editing sites that pass threshold

In [77]:
# clean up dfThr for export
dfThrClean=dfThr[['A','G','Treatment','wpi','wpiPosition']]


In [78]:
# test whether at least 2 replicates of non-edited site exist
uniqueSites=dfThrClean['wpiPosition'].unique()
excludeSites=[]
for i in range(0,len(uniqueSites)):
    uS=dfThrClean.loc[dfThrClean['wpiPosition']==uniqueSites[i]]
    if len(uS['Treatment'].loc[uS['Treatment']=="NBH"]) < 2:
        #print("In wpiPosition {} treated with NBH less than 2 replicates".format(uniqueSites[i]))
        excludeSites.append(uniqueSites[i])
    if len(uS['Treatment'].loc[uS['Treatment']=="RML6"]) < 2:
        #print("In wpiPosition {} treated with RML6 less than 2 replicates".format(uniqueSites[i]))
         excludeSites.append(uniqueSites[i])

In [79]:
dfThrClean=dfThrClean.loc[~dfThrClean['wpiPosition'].isin(excludeSites)]
dfThrClean.to_csv('muscleRediEli.csv') # export dfThrClean

In [80]:
excludeSites

['14+75719719',
 '18+7936048',
 '18+7936048',
 '4+7936048',
 '4+7936048',
 '4+77407731',
 '4+77407731']

In [81]:
# clean up dfThr for export
#dfThrClean=dfThr[['A','G','Treatment','wpi','wpiPosition']]
# only on 1 replicate: 4+7936048, 4+77407731, only on 4: 14+74226862, 14+75719719; only on 2: 18+7936048
#dfThrClean.to_csv('/Users/karlfrontzek/Documents/UCL/rna_editing/prion/muscleRediEli.csv')


#### analyse csv using R, see README.md
#### import csv from R below

In [82]:
# load p-Value adjusted results from Redit


In [83]:
# analyse statistical testing with Redit, see R
dfP=pd.read_csv('/Users/karlfrontzek/Documents/UCL/rna_editing/prionREDI/muscleEditingSitesRedit.csv')

In [84]:
dfP[['wpi','site1']]=dfP['site'].str.split("+",expand=True)

In [85]:
from statsmodels.stats.multitest import multipletests # for fdr

In [86]:
wpiUnique=dfP['wpi'].unique()

In [87]:
# adjust for fdr < 0.05 
# preassign dataframe
tT=pd.DataFrame({'wpi':[],'sig':[],'fdr':[],'pValue':[],'index':[]})
for i in range(0,len(wpiUnique)): # loop through all wpi
    tTtemp=pd.DataFrame({'wpi':[],'sig':[],'fdr':[],'pValue':[],'index':[]}) # preassign temp datafram
    # add sig and pVal to dataframe
    tTtemp['sig']=multipletests(dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]],alpha=0.05,method="fdr_bh")[0]
    tTtemp['fdr']=multipletests(dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]],alpha=0.05,method="fdr_bh")[1]
    tTtemp[['index','pValue']]=dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]].reset_index()
    tTtemp['wpi']=wpiUnique[i]
    tT=tT.append(tTtemp)

In [88]:
bloodRediAllEvents=tT.merge(dfP,left_on="pValue",right_on="pValue") # merge sites and fdr on "pValue"

In [89]:
bloodRediAllEvents.loc[bloodRediAllEvents['sig']==1] # show significant editing events

,wpi_x,sig,fdr,pValue,index,site,wpi_y,site1


# spleen

In [90]:
# get files from folder
muscleFolder="/Users/karlfrontzek/Documents/UCL/rna_editing/prionREDI/spleen"
os.chdir(muscleFolder)
onlyfiles = [f for f in os.listdir(muscleFolder) if os.path.isfile(os.path.join(muscleFolder, f))]

In [91]:
# read files in dataframe
df = pd.DataFrame()
df0 = pd.DataFrame()
for i in range(0,len(onlyfiles)):
    fname=onlyfiles[i].split(".")[0] # assign sample name
    df0=pd.read_csv(onlyfiles[i], sep="\t") # read file
    df0['sampleID']=fname # assign sample to table
    df=df.append(df0) # append table

In [92]:
# split base counts
dfb=df["BaseCount[A,C,G,T]"].str.split(",",expand=True) # expand column
dfb.columns=['A',"C","G","T"] # rename columns
dfb['A']=dfb['A'].str.replace('[',"") # replace brackets
dfb['T']=dfb['T'].str.replace(']',"")

In [93]:
df[['A',"C","G","T"]]=dfb # merge tables

In [94]:
df[['A',"C","G","T"]]= df[['A',"C","G","T"]].astype("int")

In [95]:
df.loc[df['T']>=3]# 1 site with 3xT, but A>T, not C>G site due to 172xA->807xG

,Region,Position,Reference,Strand,Coverage-q30,MeanQ,"BaseCount[A,C,G,T]",AllSubs,Frequency,sampleID,A,C,G,T
39,chr5,77407731,A,0,982,53.95,"[172, 0, 807, 3]",AG,0.82,PB-23,172,0,807,3


In [96]:
# keep only G>=3
df_raw=df
df=df.loc[df["G"]>=3]

In [97]:
# merge metadata
os.chdir("/Users/karlfrontzek/Documents/UCL/rna_editing/prion/")
# read meta-data and extract ID
peri_meta=pd.read_csv("meta_p3506_PeripheralSamples.csv",sep=";")
peri_meta.columns=['TubeID','SampleID','Treatment','wpi','Region']
peri_meta=peri_meta.loc[peri_meta['Region']=="spleen"]
peri_meta['SampleID']=peri_meta['SampleID'].str.split("_",expand=True)[0]

In [98]:
# calculate editing frequency
df['editFreq']=df['G']/df['A']
# sort by editing frequency >= 1%
#len(df['editFreq'])
#len(df.loc[df['editFreq']]) # 2 values are below 0.1%
df=df.loc[df['editFreq']>= 0.01]

/Users/karlfrontzek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [99]:
df=df.merge(peri_meta,left_on="sampleID",right_on="SampleID") # merge tables

In [100]:
# keep df_raw for statistical comparisons
df_raw=df_raw.merge(peri_meta,left_on="sampleID",right_on="SampleID") # merge tables

In [101]:
# export raw data for supplementary table, e.g. cog3 terminal stage
df_raw.loc[(df_raw['wpi']=="16") & (df_raw['Position']==7936048.0)].to_csv("rediFlnbSuppTab3.csv")
df_raw.loc[(df_raw['wpi']=="16") & (df_raw['Position']==172092348.0)].to_csv("rediCopaSuppTab3.csv")

In [102]:
# test if edited site occurs at least in 2/3 of replicates, easy: positions are all unique
occurence = []
wpii=[4,4,8,8,12,12,14,14,16,16,18,18,20,20,'term','term']
treat=['RML6','NBH']*8
loopi=pd.DataFrame({"treat":treat,"wpi":wpii})

In [103]:
# extract sites that are edited at least in 2/3 replicates

testSites=pd.DataFrame([])
for i in range(0,len(loopi)):
    tS=pd.DataFrame([])
    tS[['Position','Counts']]=pd.DataFrame(df['Position'].loc[(df['wpi']==str(loopi['wpi'].iloc[i])) & (df['Treatment']==loopi['treat'].iloc[i])].value_counts()).reset_index()
    tS['wpi']=loopi['wpi'].iloc[i]
    tS['Treatment']=loopi['treat'].iloc[i]
    testSites=testSites.append(tS)

In [104]:
dfReplicates=df[['wpi','Treatment','SampleID']].drop_duplicates()
totalRep=pd.Series(list(zip(dfReplicates.wpi, dfReplicates.Treatment))).value_counts()
totalRep=totalRep.reset_index()
totalRep.columns=['Index','Rep']

In [105]:
testSites['wpi']=testSites['wpi'].astype('str')

In [106]:
testSites['wpiTreatment']=list(zip(testSites.wpi, testSites.Treatment)) # merge wpi+Treatment for threshold 
Thr=pd.DataFrame({'Index':[],'Thr':[]})                                                       # calculation of edited sites
for i in range(0,len(testSites)): # iterate over testsite counts, keep those occuring in floor(2/3 of samples)
    wpiTreat=testSites['wpiTreatment'].iloc[i]  # read current treatment+wpi combi
    Tr=math.floor(totalRep['Rep'].loc[totalRep['Index']==wpiTreat]*2/3) # calculate current threshold
    Tp=pd.DataFrame({'Index':str(i),'Thr':str(Tr)},index=[i]) # make list of thresholds per edited site
    Thr=Thr.append(Tp)
#testSitesThr=testSites['Position'].loc[testSites['Counts']!=1] # extract test sites 
#testSites['Counts'].loc[testSites['Position'].isin(testSites1)].head(50)

In [107]:
testSitesThr=testSites.reset_index().join(Thr) # join TestSites and Threshold

In [108]:
testSitesThr['Counts']=testSitesThr['Counts'].astype('int')
testSitesThr['Thr']=testSitesThr['Thr'].astype('int') # convert counts to int

In [109]:
testSitesThr=testSitesThr.loc[testSitesThr['Counts']>=testSitesThr['Thr']] # select for editing events which pass threshold

In [110]:
# make combination of position+wpi
uniqueEdSitesThr=pd.Series(list(zip(testSitesThr.wpi, testSitesThr.Position)))
uniqueEdSitesThr=uniqueEdSitesThr.astype('str') # convert tuple to str

In [111]:
df_raw['wpiPosition']=list(zip(df_raw.wpi, df_raw.Position)) # combine wpi+Position in df_raw
df_raw['wpiPosition']=df_raw['wpiPosition'].astype('str') # convert tuple to str

In [112]:
testSitesThr['wpiPosition']=testSitesThr['wpi']+'+'+testSitesThr['Position'].astype('str')
df_raw['wpiPosition']=df_raw['wpi']+'+'+df_raw['Position'].astype('str')

In [113]:
uniqueTestSitesThr=testSitesThr['wpiPosition'].unique() # retrieve unique editing sites

In [114]:
uniqueTestSitesThr=list(uniqueTestSitesThr) # convert unique editing sites to list

In [115]:
dfThr=df_raw.loc[df_raw['wpiPosition'].isin(uniqueTestSitesThr)] # keep editing sites that pass threshold

In [116]:
# clean up dfThr for export
dfThrClean=dfThr[['A','G','Treatment','wpi','wpiPosition']]


In [117]:
# test whether at least 2 replicates of non-edited site exist
uniqueSites=dfThrClean['wpiPosition'].unique()
excludeSites=[]
for i in range(0,len(uniqueSites)):
    uS=dfThrClean.loc[dfThrClean['wpiPosition']==uniqueSites[i]]
    if len(uS['Treatment'].loc[uS['Treatment']=="NBH"]) < 2:
        #print("In wpiPosition {} treated with NBH less than 2 replicates".format(uniqueSites[i]))
        excludeSites.append(uniqueSites[i])
    if len(uS['Treatment'].loc[uS['Treatment']=="RML6"]) < 2:
        #print("In wpiPosition {} treated with RML6 less than 2 replicates".format(uniqueSites[i]))
         excludeSites.append(uniqueSites[i])

In [118]:
dfThrClean=dfThrClean.loc[~dfThrClean['wpiPosition'].isin(excludeSites)]
dfThrClean.to_csv('spleenRediEli.csv') # export dfThrClean

#### analyse csv using R, see README.md
#### import csv from R below

In [119]:
# load p-Value adjusted results from Redit

In [120]:
# analyse statistical testing with Redit, see R
dfP=pd.read_csv('/Users/karlfrontzek/Documents/UCL/rna_editing/prionREDI/spleenEditingSitesRedit.csv')

In [121]:
dfP[['wpi','site1']]=dfP['site'].str.split("+",expand=True)

In [122]:
from statsmodels.stats.multitest import multipletests # for fdr

In [123]:
wpiUnique=dfP['wpi'].unique()

In [124]:
# adjust for fdr < 0.05 
# preassign dataframe
tT=pd.DataFrame({'wpi':[],'sig':[],'fdr':[],'pValue':[],'index':[]})
for i in range(0,len(wpiUnique)): # loop through all wpi
    tTtemp=pd.DataFrame({'wpi':[],'sig':[],'fdr':[],'pValue':[],'index':[]}) # preassign temp datafram
    # add sig and pVal to dataframe
    tTtemp['sig']=multipletests(dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]],alpha=0.05,method="fdr_bh")[0]
    tTtemp['fdr']=multipletests(dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]],alpha=0.05,method="fdr_bh")[1]
    tTtemp[['index','pValue']]=dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]].reset_index()
    tTtemp['wpi']=wpiUnique[i]
    tT=tT.append(tTtemp)

In [125]:
bloodRediAllEvents=tT.merge(dfP,left_on="pValue",right_on="pValue") # merge sites and fdr on "pValue"

In [126]:
bloodRediAllEvents.loc[bloodRediAllEvents['sig']==1] # show significant editing events

,wpi_x,sig,fdr,pValue,index,site,wpi_y,site1
5,16,1.0,0.045061,0.005688,5.0,16+7936048,16,7936048
7,16,1.0,0.045061,0.006932,7.0,16+172092348,16,172092348


In [127]:
#df_raw
df_raw.loc[(df_raw['Position']==7936048) & (df_raw['wpi']=="16")]

,Region_x,Position,Reference,Strand,Coverage-q30,MeanQ,"BaseCount[A,C,G,T]",AllSubs,Frequency,sampleID,A,C,G,T,TubeID,SampleID,Treatment,wpi,Region_y,wpiPosition
18,chr14,7936048,A,1,32,45.06,"[32, 0, 0, 0]",-,0.00,PB-79,32,0,0,0,p1608_4211/80,PB-79,NBH,16,spleen,16+7936048
174,chr14,7936048,A,1,39,45.54,"[28, 0, 11, 0]",AG,0.28,PB-78,28,0,11,0,p1608_4211/26,PB-78,NBH,16,spleen,16+7936048
1350,chr14,7936048,A,1,47,38.57,"[45, 0, 2, 0]",-,0.00,PB-71,45,0,2,0,p1608_4211/8,PB-71,NBH,16,spleen,16+7936048
1431,chr14,7936048,A,1,17,45.71,"[9, 0, 8, 0]",AG,0.47,PB-73,9,0,8,0,p1608_4211/79,PB-73,RML6,16,spleen,16+7936048
1594,chr14,7936048,A,1,46,48.80,"[28, 0, 18, 0]",AG,0.39,PB-66,28,0,18,0,p1608_4211/7,PB-66,RML6,16,spleen,16+7936048
1821,chr14,7936048,A,1,24,41.12,"[12, 0, 12, 0]",AG,0.50,PB-75,12,0,12,0,p1608_4211/25,PB-75,RML6,16,spleen,16+7936048


In [128]:
df_raw.loc[(df_raw['Position']==172092348) & (df_raw['wpi']=="16")]

,Region_x,Position,Reference,Strand,Coverage-q30,MeanQ,"BaseCount[A,C,G,T]",AllSubs,Frequency,sampleID,A,C,G,T,TubeID,SampleID,Treatment,wpi,Region_y,wpiPosition
32,chr1,172092348,A,1,318,38.36,"[283, 0, 35, 0]",AG,0.11,PB-79,283,0,35,0,p1608_4211/80,PB-79,NBH,16,spleen,16+172092348
187,chr1,172092348,A,1,341,38.16,"[304, 0, 37, 0]",AG,0.11,PB-78,304,0,37,0,p1608_4211/26,PB-78,NBH,16,spleen,16+172092348
1364,chr1,172092348,A,1,477,37.70,"[432, 0, 45, 0]",-,0.00,PB-71,432,0,45,0,p1608_4211/8,PB-71,NBH,16,spleen,16+172092348
1443,chr1,172092348,A,1,361,38.23,"[335, 0, 26, 0]",-,0.00,PB-73,335,0,26,0,p1608_4211/79,PB-73,RML6,16,spleen,16+172092348
1606,chr1,172092348,A,1,332,38.34,"[322, 0, 10, 0]",-,0.00,PB-66,322,0,10,0,p1608_4211/7,PB-66,RML6,16,spleen,16+172092348
1833,chr1,172092348,A,1,311,38.19,"[296, 0, 15, 0]",-,0.00,PB-75,296,0,15,0,p1608_4211/25,PB-75,RML6,16,spleen,16+172092348


# brain

In [129]:
# get files from folder
muscleFolder="/Users/karlfrontzek/Documents/UCL/rna_editing/prionREDI/brain"
os.chdir(muscleFolder)
onlyfiles = [f for f in os.listdir(muscleFolder) if os.path.isfile(os.path.join(muscleFolder, f))]

In [130]:
# read files in dataframe
df = pd.DataFrame()
df0 = pd.DataFrame()
for i in range(0,len(onlyfiles)):
    fname=onlyfiles[i].split("A-")[1] # assign sample name
    df0=pd.read_csv(onlyfiles[i], sep="\t") # read file
    df0['sampleID']=fname[0:-4] # assign sample to table
    df=df.append(df0) # append table

In [131]:
# split base counts
dfb=df["BaseCount[A,C,G,T]"].str.split(",",expand=True) # expand column
dfb.columns=['A',"C","G","T"] # rename columns
dfb['A']=dfb['A'].str.replace('[',"") # replace brackets
dfb['T']=dfb['T'].str.replace(']',"")

In [132]:
df[['A',"C","G","T"]]=dfb # merge tables

In [133]:
df[['A',"C","G","T"]]= df[['A',"C","G","T"]].astype("int")

In [134]:
df.loc[df['T']>=3]# 3 sites with 3xT, but A>T, not C>G site due to 172xA->807xG

,Region,Position,Reference,Strand,Coverage-q30,MeanQ,"BaseCount[A,C,G,T]",AllSubs,Frequency,sampleID,A,C,G,T
38,chr3,80706912,A,0,734,41.89,"[0, 0, 730, 4]",AG AT,1.00,PB-15,0,0,730,4
9,chr11,46272643,A,0,996,40.26,"[208, 0, 784, 4]",AG,0.79,PB-43,208,0,784,4
9,chr11,46272643,A,0,1176,40.08,"[241, 0, 931, 4]",AG,0.79,PB-27,241,0,931,4


In [135]:
# keep only G>=3
df_raw=df
df=df.loc[df["G"]>=3]

In [136]:
# merge metadata
os.chdir("/Users/karlfrontzek/Documents/UCL/rna_editing/prionREDI/")
# read meta-data and extract ID
peri_meta=pd.read_csv("brainMetaMain.csv",sep=";")

peri_meta.columns=['TubeID','SampleID','Genotype','Treatment','wpi','Region','Cohort']
peri_meta['SampleID']=peri_meta['SampleID'].str.split("_",expand=True)[0]


In [137]:
# calculate editing frequency
df['editFreq']=df['G']/df['A']
# sort by editing frequency >= 1%
#len(df['editFreq'])
#len(df.loc[df['editFreq']]) # 2 values are below 0.1%
df=df.loc[df['editFreq']>= 0.01]

/Users/karlfrontzek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [138]:
df.merge(peri_meta,left_on="sampleID",right_on="SampleID")

,Region_x,Position,Reference,Strand,Coverage-q30,MeanQ,"BaseCount[A,C,G,T]",AllSubs,Frequency,sampleID,...,G,T,editFreq,TubeID,SampleID,Genotype,Treatment,wpi,Region_y,Cohort
0,chrX,41654252,A,1,231,41.18,"[37, 0, 194, 0]",AG,0.84,PB-15,...,194,0,5.243243,p1608_2530/5,PB-15,wt,NBH,4,Hippocampus,main
1,chrX,72445292,A,0,62,47.69,"[10, 0, 52, 0]",AG,0.84,PB-15,...,52,0,5.200000,p1608_2530/5,PB-15,wt,NBH,4,Hippocampus,main
2,chrX,74226862,A,0,58,46.93,"[18, 0, 40, 0]",AG,0.69,PB-15,...,40,0,2.222222,p1608_2530/5,PB-15,wt,NBH,4,Hippocampus,main
3,chrX,147169686,A,1,20,48.55,"[1, 0, 19, 0]",AG,0.95,PB-15,...,19,0,19.000000,p1608_2530/5,PB-15,wt,NBH,4,Hippocampus,main
4,chrX,147169688,A,1,20,48.35,"[1, 0, 19, 0]",AG,0.95,PB-15,...,19,0,19.000000,p1608_2530/5,PB-15,wt,NBH,4,Hippocampus,main
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,chr2,157558140,A,0,153,57.10,"[121, 0, 32, 0]",AG,0.21,PB-30,...,32,0,0.264463,p1608_2530/10,PB-30,wt,NBH,8,Hippocampus,main
1792,chr2,157558149,A,0,152,56.24,"[107, 0, 45, 0]",AG,0.30,PB-30,...,45,0,0.420561,p1608_2530/10,PB-30,wt,NBH,8,Hippocampus,main
1793,chr1,66672714,A,1,146,40.41,"[113, 0, 33, 0]",AG,0.23,PB-30,...,33,0,0.292035,p1608_2530/10,PB-30,wt,NBH,8,Hippocampus,main
1794,chr1,172092348,A,1,411,40.27,"[390, 0, 21, 0]",-,0.00,PB-30,...,21,0,0.053846,p1608_2530/10,PB-30,wt,NBH,8,Hippocampus,main


In [139]:
df=df.merge(peri_meta,left_on="sampleID",right_on="SampleID") # merge tables

In [140]:
# keep df_raw for statistical comparisons
df_raw=df_raw.merge(peri_meta,left_on="sampleID",right_on="SampleID") # merge tables

In [141]:
# test if edited site occurs at least in 2/3 of replicates, easy: positions are all unique
occurence = []
wpii=[4,4,8,8,12,12,14,14,16,16,18,18,20,20,'term','term']
treat=['RML6','NBH']*8
loopi=pd.DataFrame({"treat":treat,"wpi":wpii})

In [142]:
# extract sites that are edited at least in 2/3 replicates

testSites=pd.DataFrame([])
for i in range(0,len(loopi)):
    tS=pd.DataFrame([])
    tS[['Position','Counts']]=pd.DataFrame(df['Position'].loc[(df['wpi']==str(loopi['wpi'].iloc[i])) & (df['Treatment']==loopi['treat'].iloc[i])].value_counts()).reset_index()
    tS['wpi']=loopi['wpi'].iloc[i]
    tS['Treatment']=loopi['treat'].iloc[i]
    testSites=testSites.append(tS)

In [144]:
dfReplicates=df[['wpi','Treatment','SampleID']].drop_duplicates()
totalRep=pd.Series(list(zip(dfReplicates.wpi, dfReplicates.Treatment))).value_counts()
totalRep=totalRep.reset_index()
totalRep.columns=['Index','Rep']

In [145]:
testSites['wpi']=testSites['wpi'].astype('str')

In [146]:
testSites['wpiTreatment']=list(zip(testSites.wpi, testSites.Treatment)) # merge wpi+Treatment for threshold 
Thr=pd.DataFrame({'Index':[],'Thr':[]})                                                       # calculation of edited sites
for i in range(0,len(testSites)): # iterate over testsite counts, keep those occuring in floor(2/3 of samples)
    wpiTreat=testSites['wpiTreatment'].iloc[i]  # read current treatment+wpi combi
    Tr=math.floor(totalRep['Rep'].loc[totalRep['Index']==wpiTreat]*2/3) # calculate current threshold
    Tp=pd.DataFrame({'Index':str(i),'Thr':str(Tr)},index=[i]) # make list of thresholds per edited site
    Thr=Thr.append(Tp)
#testSitesThr=testSites['Position'].loc[testSites['Counts']!=1] # extract test sites 
#testSites['Counts'].loc[testSites['Position'].isin(testSites1)].head(50)

In [147]:
testSitesThr=testSites.reset_index().join(Thr)# join TestSites and Threshold

In [148]:
testSitesThr['Counts']=testSitesThr['Counts'].astype('int')
testSitesThr['Thr']=testSitesThr['Thr'].astype('int') # convert counts to int

In [149]:
testSitesThr

,index,Position,Counts,wpi,Treatment,wpiTreatment,Index,Thr
0,0,12411582,3,4,RML6,"(4, RML6)",0,2
1,1,75719719,3,4,RML6,"(4, RML6)",1,2
2,2,74226862,3,4,RML6,"(4, RML6)",2,2
3,3,7936048,3,4,RML6,"(4, RML6)",3,2
4,4,32047432,3,4,RML6,"(4, RML6)",4,2
...,...,...,...,...,...,...,...,...
700,44,23960539,1,term,NBH,"(term, NBH)",700,2
701,45,3415084,1,term,NBH,"(term, NBH)",701,2
702,46,23960622,1,term,NBH,"(term, NBH)",702,2
703,47,49244334,1,term,NBH,"(term, NBH)",703,2


In [150]:
testSitesThr=testSitesThr.loc[testSitesThr['Counts']>=testSitesThr['Thr']] # select for editing events which pass threshold

In [151]:
# make combination of position+wpi
uniqueEdSitesThr=pd.Series(list(zip(testSitesThr.wpi, testSitesThr.Position)))
uniqueEdSitesThr=uniqueEdSitesThr.astype('str') # convert tuple to str

In [158]:
df_raw.columns=['Region_x', 'Position', 'Reference', 'Strand', 'Coverage-q30', 'MeanQ',
       'BaseCount[A,C,G,T]', 'AllSubs', 'Frequency', 'sampleID', 'A', 'C', 'G',
       'T', 'TubeID', 'SampleID', 'Genotype', 'Treatment', 'wpi_y', 'Region_y',
       'Cohort', 'wpiPosition'] #rename wpi -> wpi_y

In [159]:
df_raw['wpiPosition']=list(zip(df_raw.wpi_y, df_raw.Position)) # combine wpi+Position in df_raw
df_raw['wpiPosition']=df_raw['wpiPosition'].astype('str') # convert tuple to str

In [161]:
testSitesThr['wpiPosition']=testSitesThr['wpi']+'+'+testSitesThr['Position'].astype('str')
df_raw['wpiPosition']=df_raw['wpi_y']+'+'+df_raw['Position'].astype('str')

/Users/karlfrontzek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [162]:
uniqueTestSitesThr=testSitesThr['wpiPosition'].unique() # retrieve unique editing sites

In [163]:
uniqueTestSitesThr=list(uniqueTestSitesThr) # convert unique editing sites to list

In [164]:
dfThr=df_raw.loc[df_raw['wpiPosition'].isin(uniqueTestSitesThr)] # keep editing sites that pass threshold

In [167]:
# clean up dfThr for export
dfThrClean=dfThr[['A','G','Treatment','wpi_y','wpiPosition']]


In [170]:
# test whether at least 2 replicates of non-edited site exist
uniqueSites=dfThrClean['wpiPosition'].unique()
excludeSites=[]
for i in range(0,len(uniqueSites)):
    uS=dfThrClean.loc[dfThrClean['wpiPosition']==uniqueSites[i]]
    if len(uS['Treatment'].loc[uS['Treatment']=="NBH"]) < 2:
        #print("In wpiPosition {} treated with NBH less than 2 replicates".format(uniqueSites[i]))
        excludeSites.append(uniqueSites[i])
    if len(uS['Treatment'].loc[uS['Treatment']=="RML6"]) < 2:
        #print("In wpiPosition {} treated with RML6 less than 2 replicates".format(uniqueSites[i]))
         excludeSites.append(uniqueSites[i])

In [171]:
dfThrClean=dfThrClean.loc[~dfThrClean['wpiPosition'].isin(excludeSites)]
dfThrClean.to_csv('brainRediEli.csv') # export dfThrClean

#### analyse csv using R, see README.md
#### import csv from R below

In [ ]:
# load p-Value adjusted results from Redit

In [172]:
# analyse statistical testing with Redit, see R
dfP=pd.read_csv('/Users/karlfrontzek/Documents/UCL/rna_editing/prionREDI/brainEditingSitesRedit.csv')

In [173]:
dfP[['wpi','site1']]=dfP['site'].str.split("+",expand=True)

In [174]:
from statsmodels.stats.multitest import multipletests # for fdr

In [175]:
wpiUnique=dfP['wpi'].unique()

In [176]:
# adjust for fdr < 0.05 
# preassign dataframe
tT=pd.DataFrame({'wpi':[],'sig':[],'fdr':[],'pValue':[],'index':[]})
for i in range(0,len(wpiUnique)): # loop through all wpi
    tTtemp=pd.DataFrame({'wpi':[],'sig':[],'fdr':[],'pValue':[],'index':[]}) # preassign temp datafram
    # add sig and pVal to dataframe
    tTtemp['sig']=multipletests(dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]],alpha=0.05,method="fdr_bh")[0]
    tTtemp['fdr']=multipletests(dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]],alpha=0.05,method="fdr_bh")[1]
    tTtemp[['index','pValue']]=dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]].reset_index()
    tTtemp['wpi']=wpiUnique[i]
    tT=tT.append(tTtemp)

In [177]:
bloodRediAllEvents=tT.merge(dfP,left_on="pValue",right_on="pValue") # merge sites and fdr on "pValue"

In [178]:
bloodRediAllEvents.loc[bloodRediAllEvents['sig']==1] # show significant editing events

,wpi_x,sig,fdr,pValue,index,site,wpi_y,site1
